In [ ]:
import tensorflow as tf
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
tf.keras.backend.set_floatx('float64')
tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)

In [ ]:
df=pd.read_csv("nation_level_daily_latest.csv")
df=df.fillna(method ='pad') 
df.head()

In [ ]:
features=["Total Confirmed","Total Recovered","Total Deceased"]
df=df[features]

In [ ]:
series=df.values[::]
series=series.astype("float64")
series.shape

In [ ]:
def make_data(series,window_size):
    
    data=[]
    labels=[]

    for i in range(len(series)-window_size):
        temp=series[i:i+window_size+1]
        data.append(temp[:window_size])
        labels.append(temp[window_size:,0])

    return np.array(data),np.array(labels)

In [ ]:
split=int(len(series)*0.9)
train_series=series[:split]
test_series=series[split:]

In [ ]:
batch_size=32
num_epochs=10000
window_size=7

In [ ]:
train_data,train_labels=make_data(train_series,window_size)
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
train_dataset=train_dataset.batch(batch_size)

In [ ]:
test_data,test_labels=make_data(test_series,window_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_labels))
test_dataset=test_dataset.batch(batch_size)

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_loss')<0.2):
            print("\nCancelling training!")
            self.model.stop_training = True

In [ ]:
callbacks = myCallback()

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(128,activation="relu", return_sequences=True),
    tf.keras.layers.LSTM(128,activation="relu", return_sequences=True),
    tf.keras.layers.GRU(128,activation="relu", return_sequences=True),
    tf.keras.layers.GRU(128,activation="relu", return_sequences=False), 
    tf.keras.layers.Dense(32,activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1),
])

In [ ]:
model.compile(loss=tf.keras.losses.MeanAbsolutePercentageError(),
              optimizer=tf.keras.optimizers.Adam())

In [ ]:
history = model.fit(train_dataset, epochs=num_epochs,validation_data=test_dataset, callbacks=[callbacks])

In [ ]:
pred=model.predict(test_data)
pred.shape

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(pred,label="Predicted")
plt.plot(test_labels,label="Actual")
plt.xlabel("Time in Days from June 12 to June 19")
plt.ylabel("Total Confirmed Cases")
plt.legend()
plt.title("Comparison of Actual and Predicted Total Confirmed cases")
plt.grid(True)
plt.show()
plt.savefig("LSTM-GRU-National-Result.png")

In [ ]:
tf.keras.metrics.mean_absolute_percentage_error(test_labels,pred).numpy().mean()

In [ ]:
# tf.keras.models.save_model(model,'saved_model/national_model',save_format='h5') 